In [ ]:
import gym, ray
import numpy as np
import copy
import requests
from ray.rllib.agents.ppo import PPOTrainer

Note:

- This is a two-player game. So, maybe we could fix one player to always do the same thing, and then learn against that?
- Or, we could learn two agents?
- Or ???

In [ ]:
class IteratedPDEnv(gym.Env):
    
    INITIAL_STATE = {"totalPlayer2": 0, "totalPlayer1": 0}
    
    state = None
    done  = False
    
    def __init__ (self, env_config):
        # Note: Our action space is for ONE player; namely, exactly two
        # choices:
        #  - Defect
        #  - Cooperate
        self.action_space      = gym.spaces.Discrete(2)
        
        # We make no observations.
        self.observation_space = gym.spaces.Discrete(1)
        
        """
        self.observation_space = gym.spaces.Box(
                                      low=0
                                    , high=np.iinfo(np.int32).max
                                    , shape=(1,)
                                    , dtype=np.int32
                                    )
        """
        
        # self.seed(1)
        self.reset()
    
    
    def reset (self):
        self.state = copy.deepcopy(self.INITIAL_STATE)
        self.done = False
        return 0
    
    
    def step (self, action):
        # action is either 0 or 1.
        
        if action == 0:
            player1Action = "Cooperate"
        else:
            player1Action = "Defect"
        
        assert action in [0, 1], "Unknown action!"
        
        # We are done if someone spent >100 years in jail.
        if self.state["totalPlayer1"] >= 100:
            self.done = True
            
        if self.state["totalPlayer2"] >= 100:
            self.done = True
        
        data = { "player1Action": player1Action
               , "player2Action": "Cooperate" # Goody-two-shoes
               , "startingState": self.state
               }
        
        # Do a post to the server; get the payoffs.
        response = requests.post("http://localhost:3000/play", json=data).json()
        
        # Update the state.
        self.state["totalPlayer1"] += response["lastPlayer1Payoff"]
        self.state["totalPlayer2"] += response["lastPlayer2Payoff"]
        
        obs = response["lastPlayer1Payoff"]
        
        # TODO: The reward should NOT be the amount of time in jail; it should
        # the DIFFERENCE
        
        # reward = 10 - 6
        # reward = 4 -> Good for me
        
        # reward = 1 - 19
        # reward = -18 -> Bad for me.
        reward = self.state["totalPlayer2"] - self.state["totalPlayer1"]
        
        return [ 0, reward, self.done, response ]

In [ ]:
env = IteratedPDEnv(env_config = {})

In [ ]:
env.step(action=1)

### Let's try training it!

In [ ]:
ray.init(local_mode=True)

In [ ]:
trainer = PPOTrainer(env=IteratedPDEnv, config={
    "framework": "tf2",
    "num_workers": 1,
    "env_config": {},
    "create_env_on_driver": True
})

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()